### Intro

This note book will organize the galaxy catalogs from the UniverseMachine model using the small multidark planck simulation (SMDPL).  The results are provided by Peter Behroozi, and the initial catalogs are made by Christopher Bradshaw. 

The models are for galaxies at z~0.4.  Two models are provided: 
1. `sfr_catalog_0.712400.txt.gz`: old fashion `bcg` and `icl` components one. 
2. `sfr_catalog_insitu_exsitu_0.712400.txt.gz`: new `in-situ` and `ex-situ` components one. 
3. `particles_0.712400.txt.gz`: DM particles file. 

The useful information has been summarized into a smaller catalog, also the central and satellite galaxies are separated now.  (`.npz`) format. 

Here, we: 

1. Explore the basic properties of these new models. 
2. Compare their SMF with observed ones. 
3. Explore new M100-M10 models.
4. Select the relevant massive galaxies, and save the value-added catalog. 

We also need to downsample the particle tables and pre-compute the `DeltaSigma` signals around the selected massive galaxies.

#### Small MultiDark Simulation

* For details, please see: https://www.cosmosim.org/cms/simulations/smdpl/

![](cosmosim_smdpl_sum.png)

In [1]:
%matplotlib inline
%load_ext autoreload 
%autoreload 2

import os
import sys
import copy 
from time import time

import numpy as np 
import matplotlib.pyplot as plt
    
import matplotlib.mlab as ml
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.ticker import MaxNLocator

%config InlineBackend.figure_format='retina'
plt.rcParams['figure.dpi'] = 100.0
plt.rcParams['figure.facecolor'] = 'w'
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.size'] = 12.0
plt.rc('text', usetex=True)

# Personal repo
from stellar_mass_function import compute_smf, \
    bootstrap_smf
    
from hsc_massive import \
    s16a_path, \
    sample_selection, \
    prepare_sed, \
    catalog_summary, \
    smhm, \
    plotting

In [2]:
from astropy.table import Table, Column

# Summary catalogs for SMDPL-UniverseMachine model

In [4]:
# Setup environment 
envir = s16a_path.set_env(version='astro5')
#envir = s16a_path.set_env(version='kungpao')

# Folder for new UM model 
um_dir = os.path.join(envir['s16a'], 'um2/um2_new')
um_smdpl_dir = os.path.join(um_dir, 'um_smdpl_small')

# Summary catalogs for new UM models 
um_smdpl_old = np.load(os.path.join(um_smdpl_dir, 
                                    'sfr_catalog_0.712400_final.npz'))
um_smdpl_new = np.load(os.path.join(um_smdpl_dir, 
                                    'sfr_catalog_insitu_exsitu_0.712400_final.npz'))

# Results for central and satellite galaxies
um_smdpl_old_cen = Table(um_smdpl_old['centrals'])
um_smdpl_old_sat = Table(um_smdpl_old['satellites'])

um_smdpl_new_cen = Table(um_smdpl_new['centrals'])
um_smdpl_new_sat = Table(um_smdpl_new['satellites'])

print(len(um_smdpl_old_cen), len(um_smdpl_old_sat))
print(len(um_smdpl_new_cen), len(um_smdpl_new_sat))

print(um_smdpl_old_cen.colnames)

(385125, 10809469)
(385125, 10809469)
['id', 'upid', 'x', 'y', 'z', 'm', 'mp', 'sm', 'icl', 'sfr']


In [17]:
um_smdpl_old['satellites']

array([ (       8272347655, 8272347654,   13.83487 ,   18.749399,   16.14661 ,   1.98400000e+09,   3.11700000e+09,   2.31400000e+05,   0.00000000e+00,   6.41000000e-05),
       (       8272348262, 8272347732,   15.026   ,   17.7521  ,   17.388531,   1.98400000e+09,   2.26700000e+09,   1.15200000e+03,   0.00000000e+00,   2.51000000e-06),
       (       8272348449, 8272347732,   15.09965 ,   17.864719,   17.39245 ,   4.10900000e+09,   9.77600000e+09,   3.21900000e+05,   0.00000000e+00,   1.55000000e-04),
       ...,
       (34000004223609451, 8360215083,  384.851227,  396.654236,  391.585083,   1.63100000e+10,   1.29100000e+11,   1.37000000e+09,   2.81900000e+06,   1.15700000e-02),
       (       8360215126, 8360215083,  384.742859,  396.397644,  391.56073 ,   5.68200000e+10,   2.57900000e+11,   1.36100000e+09,   2.40200000e+08,   1.95500000e-03),
       (       8360215113, 8360215083,  385.014954,  396.380035,  391.405884,   1.33400000e+12,   3.05400000e+12,   5.34800000e+10,   1.044000